Using any OpenAI model or Groq (llama 70b) model, solve the aisle mapping problem.  OpenAI code is provided here. You will do the usual 3 steps in terms of mounting the Google drive, your API key and install one of the LLM models, and import pandas.  Your goal is to do a model comparison and validation.
**The code below works for OpenAI gpt-4o model.  Have not tested it on Llama model.  Also, this code with the dataset  may blow your budget if you are not careful with the size of test dataset, so exercise caution.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install -r "/content/drive/MyDrive/LLM/Groq/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/Groq/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $GROQ_API_KEY

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


In [ ]:
from groq import Groq
client = Groq(api_key = os.getenv('GROQ_API_KEY'))
#model = 'mixtral-8x7b-32768'

In [ ]:
import pandas as pd
df = pd.read_excel('Aisle-Mapping.xlsx')

In [ ]:
groceries = df['Grocery ITEM'].dropna().tolist()
aisles = df['Aisle Category'].dropna().tolist()

In [ ]:
batch_size = 50  # Adjust the batch size as needed
separator=","

# Function to get the best matches for a batch of keywords using chat completion
def get_best_matches_batch(grocery_batch, aisle_list):
    prompt = "Match each grocery item in the  grocery list with the most appropriate aisle category from the provided list of aisle categories.\n\n"
    prompt += "The grocery list items are separated by commas. The list of aisles are also separated by commas. \n\n"
    prompt += "List of grocery items to match:\n" + "\n"+ "\n"+separator.join(grocery_batch)+  "\n\n"
    prompt += "List of provided aisle categories:\n" + "\n"+ "\n"+separator.join(aisle_list)+ "\n\n"
    prompt += "If an appropriate aisle category is not to be found in the list, use Other \n\n"
    prompt += "Return the matches in the format 'grocery item -> aisle category \n\n"
    prompt+="You must absolutely make sure that each grocery item is mapped to an aisle category"
    #print(prompt)
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that matches grocery items to aisle categories based on a typical grocery store or a supermarket in the USA."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,
        temperature=0.0
    )
    matches = response.choices[0].message.content
    return matches

In [ ]:

grocery_batch = groceries[0: batch_size]
batch_matches = get_best_matches_batch(grocery_batch, aisles)
print(batch_matches)

Here are the matches:

tropical fruit -> Produce
whole milk -> Dairy
pip fruit -> Produce
other vegetables -> Produce
whole milk -> Dairy
rolls/buns -> Bakery
other vegetables -> Produce
pot plants -> Garden
whole milk -> Dairy
tropical fruit -> Produce
citrus fruit -> Produce
beef -> Meat
frankfurter -> Meat
chicken -> Meat
butter -> Dairy
fruit/vegetable juice -> Beverages
packaged fruit/vegetables -> Canned Goods
chocolate -> Candy
specialty bar -> Snacks
other vegetables -> Produce
butter milk -> Dairy
whole milk -> Dairy
tropical fruit -> Produce
tropical fruit -> Produce
bottled water -> Beverages
yogurt -> Dairy
sausage -> Meat
other vegetables -> Produce
brown bread -> Bakery
yogurt -> Dairy
hamburger meat -> Meat
root vegetables -> Produce
pork -> Meat
beef -> Meat
pastry -> Bakery
fruit/vegetable juice -> Beverages
canned beer -> Alcohol
root vegetables -> Produce
citrus fruit -> Produce
sausage -> Meat
tropical fruit -> Produce
berries -> Produce
canned beer -> Alcohol
butte